<a href="https://colab.research.google.com/github/Coderdivine/UniqueWordDataset/blob/main/BankDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd

def extract_fields_from_xlsx(filename):
    # Read all sheets of the Excel file into a dictionary of pandas DataFrames
    all_sheets = pd.read_excel(filename, sheet_name=None)

    # Initialize variables to store extracted fields
    reference = []
    remarks = []
    transaction_details = []
    ref_no = []
    Details = []

    for sheet_name, df in all_sheets.items():
        # Iterate over the columns in the DataFrame
        print(f"*************** First Files {sheet_name} ***************")
        for column in df.columns:
            # Remove leading/trailing spaces and convert column name to lowercase
            cleaned_column = column.strip().lower()

            # Check if the cleaned column header matches the desired fields
            if 'reference' in cleaned_column:
                reference.extend(df[column].tolist())
            elif 'remarks' in cleaned_column:
                remarks.extend(df[column].tolist())
            elif 'transaction details' in cleaned_column:
                transaction_details.extend(df[column].tolist())
            elif 'ref no' in cleaned_column:
                ref_no.extend(df[column].tolist())
            elif 'details' in cleaned_column:
                Details.extend(df[column].tolist())


    # Return the extracted fields as a dictionary
    extracted_fields = {
        'Reference': reference,
        'Remarks': remarks,
        'Transaction Details': transaction_details,
        'Ref No': ref_no,
        'Details':Details
    }

    return extracted_fields

# Specify the path to the folder containing the XLSX files
folder_path = '/content/BankDatasets'

# Iterate over the files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an XLSX file
    if filename.endswith('.xlsx'):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)

        # Call the function to extract the fields
        extracted_data = extract_fields_from_xlsx(file_path)
        #print(extracted_data)

        # Access the extracted fields
        reference_list = extracted_data['Reference']
        remarks_list = extracted_data['Remarks']
        transaction_details_list = extracted_data['Transaction Details']
        ref_no_list = extracted_data['Ref No']
        details = extracted_data["Details"]
        print(extracted_data)

        # Process the extracted fields as needed
        # ...

        # Print or perform any other operations
        print(f"Processed file: {filename}")


In [ ]:
import pandas as pd
import numpy as np
import json

# Assuming you have the extracted fields stored in separate lists
transaction_details_list = extracted_data['Transaction Details']
reference_list = extracted_data['Reference']

# Create a DataFrame from the lists
df = pd.DataFrame({'sentence': transaction_details_list, 'reference': reference_list})
#df = pd.DataFrame({'transaction_details': transaction_details_list, 'reference': reference_list})

# Remove np.nan values and replace them with empty strings in transaction_details and reference columns
df['sentence'].fillna('', inplace=True)
df['reference'].fillna('', inplace=True)

# Initialize an empty list to store the unique patterns
unique_patterns = []

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    reference = row['reference']
    transaction_details = row['sentence']

    # Check if the previous and current transaction_details are different
    if index > 0 and transaction_details != df.loc[index - 1, 'sentence']:
        # Check the type of the transaction_details
        if isinstance(df.loc[index - 1, 'sentence'], str) and isinstance(transaction_details, str):
            # Find unique patterns between the previous and current transaction_details
            unique_words = list(set(df.loc[index - 1, 'sentence'].split()) - set(transaction_details.split()))
            if not unique_words:
                unique_words = [""]  # Set empty list to [""] if no unique words found
            unique_patterns.append({'sentence': transaction_details, 'unique_words': unique_words})
        else:
            # Handle non-string transaction_details by returning an empty string
            unique_patterns.append({'sentence': transaction_details, 'unique_words': [""]})
    else:
        # Handle the first row where there is no previous transaction_details to compare with
        if isinstance(transaction_details, str):
            unique_words = list(set(transaction_details.split()))
            if not unique_words:
                unique_words = [""]  # Set empty list to [""] if no unique words found
            unique_patterns.append({'sentence': transaction_details, 'unique_words': unique_words})

# Convert the list of unique patterns to JSON format
new_df_json = json.dumps(unique_patterns)
new_df = pd.DataFrame(unique_patterns)

# Print the JSON data
print(new_df_json)
print(new_df)


In [ ]:
!pip install scikit-multilearn transformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch


print(new_df['unique_words'])

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    new_df['sentence'], new_df['unique_words'], test_size=0.2, random_state=42
)

# Transforming text into numerical features using CountVectorizer
vectorizer = CountVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test).toarray()

# Converting the target variable to a binary indicator matrix
mlb = MultiLabelBinarizer()
y_train_binary = mlb.fit_transform(y_train)
y_test_binary = mlb.transform(y_test)

# Fine-tuning BERT on our specific task
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')





class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length, labels):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data.iloc[index]
        text = item['sentence']
        label = self.labels[index]

        # Tokenize the text
        inputs = self.tokenizer(
            text=text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'][0],  # Access the tokenized input_ids
            'attention_mask': inputs['attention_mask'][0],  # Access the attention mask
            'labels': torch.tensor(label, dtype=torch.float)
        }

train_dataset = CustomDataset(df.iloc[X_train.index], tokenizer, max_length=128, labels=y_train_binary)
test_dataset = CustomDataset(df.iloc[X_test.index], tokenizer, max_length=128, labels=y_test_binary)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(mlb.classes_)).to(device)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Training loop for fine-tuning BERT
model.train()
for epoch in range(5):
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} - Average Loss: {avg_loss:.4f}")

# Evaluation
model.eval()
with torch.no_grad():
    y_pred_probs = []
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        y_pred_probs.extend(torch.sigmoid(logits).cpu().numpy())

    y_pred_probs = np.array(y_pred_probs)
    y_pred_binary = (y_pred_probs >= 0.5).astype(int)

# Calculating accuracy
accuracy = accuracy_score(y_test_binary, y_pred_binary)
print("Accuracy:", accuracy)

# Calculating loss (log loss)
loss = log_loss(y_test_binary, y_pred_probs)
print("Loss:", loss)


0                                                      []
1                                                      []
2        [30-04-22, MONEY, TO, LEVY-9, TXNS:, ELEC, TRSF]
3       [AUTOTOPUP, NIGERIA, FIP:FTS/TRF/MTN, SETTLEMENT]
4                         [NELLOBYTE, SME002, /TAS220287]
                              ...                        
1064                          [UYO/TAS229742/21284/GOLDE]
1065    [from, RULE, GOLDEN, COMM, UYO/TAS229762/21284...
1066      [ICMS/5768310522192020:TAS229764/21283/GOLDENR]
1067    [ICMS/5733290522145507:TAS227700/19323/GOLDENR...
1068                           [TRANSFER, BALANCE, SWEEP]
Name: unique_words, Length: 1069, dtype: object


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['/TAS222655', '/TAS226599', '18-05-22', '24-05-22', '26-05-22', '30-04-22', 'BL/TAS228832', 'C45527/19609/OPTIMUM', 'C45642/19742/TSHABRON', 'C45713/19741/TSHABRON', 'FCSL:C46217/19461/WOLID', 'FCSL:C46222/19461/WOLID', 'FCSL:TAS221000/19941/EASY', 'FCSL:TAS221887/19941/EASY', 'FCSL:TAS222250/19941/EASY', 'FCSL:TAS222291/19941/EASY', 'FCSL:TAS223042/19941/EASY', 'FCSL:TAS223411/19941/EASY', 'FCSL:TAS223413/19941/EASY', 'FCSL:TAS224596/19941/EASY', 'FCSL:TAS224612/19941/EASY', 'FCSL:TAS225241/19941/EASY', 'FCSL:TAS225244/19941/EASY', 'FCSL:TAS225467/19941/EASY', 'FCSL:TAS226272/19941/EASY', 'FCSL:TAS226406/19386/SEAMAN', 'FCSL:TAS226772/19941/EASY', 'FCSL:TAS226774/19941/EASY', 'FCSL:TAS227197/19386/SEAMAN', 'FCSL:TAS227293/19409/STRATFORD', 'FCSL:TAS228280/19941/EASY', 'FCSL:TAS228302/19941/EASY', 'FCSL:TAS228687/19386/SEAMAN', 'FCSL:TAS229223/19941/EASY', 'FCSL:c45114/19461/WOL

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch 1 - Average Loss: 0.5652
Epoch 2 - Average Loss: 0.3077
Epoch 3 - Average Loss: 0.1750


In [ ]:
# Example sentence for prediction
new_sentence = "FIP:FTS/TRF/MTN NIGERIA AUTOTOPUP SETTLEMENT"

# Tokenize the new sentence
inputs = tokenizer(
    text=new_sentence,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Move inputs to the device
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

# Make prediction
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_probs = torch.sigmoid(logits).cpu().numpy()

# Convert probabilities to binary predictions (using threshold 0.5)
predicted_binary = (predicted_probs >= 0.5).astype(int)

# Inverse transform the binary predictions to obtain the predicted unique words
predicted_unique_words = mlb.inverse_transform(predicted_binary)

print("Predicted unique words:", predicted_unique_words)
